# Dev Notebook

# Get MAC

In [70]:
import platform, subprocess

def ping(ip: str, timeout: float = 2.0):
    param = '-n' if platform.system().lower().startswith('win') else '-c'
    cmd = ['ping', param, '1', '-w', str(timeout), ip] if param == '-n' else ['ping', param, '1', '-W', str(timeout), host]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.returncode == 0



In [74]:
import subprocess, platform, re, time

def get_mac(ip: str) -> None|str:
    ping(ip, timeout=1)
    time.sleep(1)
    system = platform.system().lower()
    cmd = ['arp', '-a', ip] if system.startswith('win') else ['arp', '-n', ip]
    try:
        output = subprocess.check_output(cmd, text=True, stderr=subprocess.DEVNULL, encoding="cp437")
    except Exception as ex:
        print(ex)
        return None
    if not (m := re.search(r'([0-9A-Fa-f]{2}[:\-]){5}[0-9A-Fa-f]{2}', output)):
        return None 
    return m.group(0).replace("-", ":").upper()


get_mac("192.168.171.32")

In [75]:
import re, socket

def send_wol_package(ip: str, mac: str, udp_port: int = 9, n: int = 3):
    mac = mac.upper()
    if not re.match(r"^([0-9A-Fa-f]{2}[:]){5}([0-9A-Fa-f]{2})$", mac):
        raise ValueError(f"Invalid MAC '{mac}'")
    mac = mac.replace(":", "")
    payload = b"\xff"*6 + bytes.fromhex(mac)*16
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
        s.settimeout(1.0)
        try:
            for i in range(n):
                s.sendto(payload, (ip, 9))
        except Exception as ex:
            print(ex)

send_wol_package("127.0.0.1", "D0:50:99:5A:8F:A3")
    

In [61]:
m.group(0)

'2c-91-ab-93-66-02'

In [1]:
import re
import socket

def send_wol_package(ip: str, mac: str, udp_port: int = 9, n: int = 3):
    mac = mac.upper()
    if not re.match(r"^([0-9A-Fa-f]{2}[:]){5}([0-9A-Fa-f]{2})$", mac):
        raise ValueError(f"Invalid MAC '{mac}'")
    mac = mac.replace(":", "")
    payload = b"\xff"*6 + bytes.fromhex(mac)*16
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
        s.settimeout(1.0)
        try:
            for i in range(n):
                s.sendto(payload, (ip, 9))
        except Exception as ex:
            print(ex)

send_wol_package("127.0.0.1", "D0:50:99:5A:8F:A3")
    

In [ ]:
def ping_host()

In [ ]:
import platform
import subprocess
import socket
import re


def ping(ip: str, timeout=5):
    """ Sends a ping packet to a host """


def ping_host(ip, timeout=2):
    """
    Ping das Ziel einmal. Liefert True wenn Ping erfolgreich (Exitcode 0), sonst False.
    timeout in Sekunden (wird an subprocess.run übergeben).
    """
    system = platform.system().lower()
    if system == 'windows':
        cmd = ['ping', '-n', '1', ip]
    else:
        # Linux / macOS: -c 1 sendet ein Paket; wir verlassen uns auf subprocess timeout.
        cmd = ['ping', '-c', '1', ip]

    try:
        # stdout/err unterdrücken
        subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, timeout=timeout, check=True)
        return True
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
        return False
    except FileNotFoundError:
        # ping nicht vorhanden (sehr unwahrscheinlich)
        return False

In [10]:
def test():
    try:
        with socket.create_connection(("192.168.11.20", 3389), timeout=1):
            return True
    except (socket.timeout, ConnectionRefusedError, OSError):
        return False
test()

False

In [7]:
socket.gethostbyname("192.168.11.")

gaierror: [Errno 11001] getaddrinfo failed

In [1]:
import platform
import subprocess
import socket
import re

MAC_RE = re.compile(r'([0-9A-Fa-f]{2}[:-]){5}[0-9A-Fa-f]{2}')

def resolve_host(host):
    """Hostname -> IP (falls nötig). Gibt IP als string zurück oder wirft OSError."""
    return socket.gethostbyname(host)

def ping_host(ip, timeout=2):
    """
    Ping das Ziel einmal. Liefert True wenn Ping erfolgreich (Exitcode 0), sonst False.
    timeout in Sekunden (wird an subprocess.run übergeben).
    """
    system = platform.system().lower()
    if system == 'windows':
        cmd = ['ping', '-n', '1', ip]
    else:
        # Linux / macOS: -c 1 sendet ein Paket; wir verlassen uns auf subprocess timeout.
        cmd = ['ping', '-c', '1', ip]

    try:
        # stdout/err unterdrücken
        subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, timeout=timeout, check=True)
        return True
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
        return False
    except FileNotFoundError:
        # ping nicht vorhanden (sehr unwahrscheinlich)
        return False

def run_cmd_capture(cmd):
    """Hilfsfunktion: führt cmd (Liste) aus und gibt stdout als str zurück oder '' bei Fehler."""
    try:
        output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, universal_newlines=True)
        return output
    except Exception:
        return ''

def find_mac_in_text(text):
    """Sucht nach einer MAC-Adresse im Text; gibt erste gefundene als normalized 'aa:bb:cc:dd:ee:ff' zurück."""
    m = MAC_RE.search(text)
    if not m:
        return None
    mac = m.group(0).lower().replace('-', ':')
    return mac

def get_mac_for_ip(ip):
    """
    Versucht verschiedene ARP / neighbor Befehle, um die MAC zur IP zu finden.
    Gibt MAC (str) oder None zurück.
    """
    system = platform.system().lower()

    # Versuche moderne Linux-Methode: ip neigh show <ip>
    if system in ('linux', 'darwin'):  # darwin (macOS) kann 'ip' fehlen; but we try
        out = run_cmd_capture(['ip', 'neigh', 'show', ip])
        mac = find_mac_in_text(out)
        if mac:
            return mac

    # Linux / macOS / Windows: arp -n <ip> oder arp <ip> oder arp -a
    out = run_cmd_capture(['arp', '-n', ip]) or run_cmd_capture(['arp', ip]) or run_cmd_capture(['arp', '-a', ip])
    mac = find_mac_in_text(out)
    if mac:
        return mac

    # Windows: arp -a und parse nach der IP
    if system == 'windows':
        out = run_cmd_capture(['arp', '-a'])
        # Suche Zeile mit IP und parse MAC danach
        # find all MACs in the same line as the ip
        for line in out.splitlines():
            if ip in line:
                mac = find_mac_in_text(line)
                if mac:
                    return mac

    # Linux alternative: arp -n ohne ip
    out = run_cmd_capture(['arp', '-n'])
    if ip in out:
        mac = find_mac_in_text(out)
        if mac:
            return mac

    # macOS alternative: arp -a
    out = run_cmd_capture(['arp', '-a'])
    if ip in out:
        mac = find_mac_in_text(out)
        if mac:
            return mac

    # Kein Eintrag gefunden
    return None

def ping_and_get_mac(host, ping_timeout=2):
    """
    Hauptfunktion: Host (Name oder IP) -> (ip, ping_ok(bool), mac_or_None, reason_str)
    """
    try:
        ip = resolve_host(host)
    except Exception as e:
        return (None, False, None, f"Hostname konnte nicht aufgelöst werden: {e}")

    ping_ok = ping_host(ip, timeout=ping_timeout)
    if not ping_ok:
        # Trotzdem versuchen, vielleicht war schon ein ARP-Eintrag vorhanden
        mac = get_mac_for_ip(ip)
        if mac:
            return (ip, False, mac, "Ping fehlgeschlagen (ICMP), aber ARP-Eintrag gefunden")
        return (ip, False, None, "Ping fehlgeschlagen; keine MAC gefunden")

    # Ping OK -> ARP/neighbor abrufen
    mac = get_mac_for_ip(ip)
    if mac:
        return (ip, True, mac, "Ping OK und MAC gefunden")
    else:
        return (ip, True, None, "Ping OK, aber kein ARP-Eintrag / MAC gefunden")

# Beispielnutzung
if __name__ == '__main__':
    test = input("Host (Name oder IP) eingeben: ").strip()
    ip, ping_ok, mac, reason = ping_and_get_mac(test, ping_timeout=3)
    print(f"IP: {ip}")
    print(f"Ping erfolgreich: {ping_ok}")
    print(f"MAC: {mac}")
    print(f"Bemerkung: {reason}")


IP: 192.168.11.21
Ping erfolgreich: False
MAC: 94:c6:91:51:9a:30
Bemerkung: Ping fehlgeschlagen (ICMP), aber ARP-Eintrag gefunden
